In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import Imputer

In [222]:
properties = pd.read_csv('data/props/datos_procesados.csv')
properties.dropna(subset=['price','property_type', 'state_name', 'place_name'], inplace=True)

/Users/mreverter/PythonProjects/datos/tp-last/env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,5,6,9,25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [223]:
train = properties.loc[:, ['property_type', 'state_name', 'place_name', 'rooms', 'expenses', 'floor', 'price']]

In [224]:
train.head(5)

,property_type,state_name,place_name,rooms,expenses,floor,price
360720,apartment,Capital Federal,Palermo,1.0,1500,NaN,71000.0
360721,apartment,Bs.As. G.B.A. Zona Sur,Temperley,NaN,270,NaN,132000.0
360722,apartment,Bs.As. G.B.A. Zona Sur,Lomas de Zamora,NaN,650,NaN,190000.0
360723,apartment,Bs.As. G.B.A. Zona Sur,Lomas de Zamora,NaN,1500,NaN,119000.0
360724,store,Bs.As. G.B.A. Zona Sur,Temperley,NaN,NaN,NaN,98000.0


In [225]:
test = train[478:]
train = train[0:478]
#test = pd.read_csv('data/test/properati_dataset_testing_noprice.csv')
test['expenses'] = test['expenses'].apply(pd.to_numeric, errors='coerce')

In [226]:
lb_make = LabelEncoder()
train['property_type_encoded'] = lb_make.fit_transform(train['property_type'])

lb_make = LabelEncoder()
train['place_name_encoded'] = lb_make.fit_transform(train['place_name'])

lb_make = LabelEncoder()
train['state_name_encoded'] = lb_make.fit_transform(train['state_name'])

In [227]:
lb_make = LabelEncoder()
test['property_type_encoded'] = lb_make.fit_transform(test['property_type'])

lb_make = LabelEncoder()
test['place_name_encoded'] = lb_make.fit_transform(test['place_name'])

lb_make = LabelEncoder()
test['state_name_encoded'] = lb_make.fit_transform(test['state_name'])

In [228]:
train_no_prices = train.loc[:, ['property_type_encoded', 'state_name_encoded', 'place_name_encoded', 'rooms', 'expenses', 'floor', 'lat', 'lon']]

/Users/mreverter/PythonProjects/datos/tp-last/env/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [229]:
test_to_predict = test.loc[:, ['property_type_encoded', 'state_name_encoded', 'place_name_encoded', 'rooms', 'expenses', 'floor']]

In [230]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(train_no_prices)
Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

train_no_prices = imp.transform(train_no_prices)

In [231]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(test_to_predict)
Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

test_to_predict = imp.transform(test_to_predict)

In [232]:
knn = neighbors.KNeighborsRegressor(25, weights="uniform", algorithm="ball_tree")

In [233]:
train_prices = train.loc[:, 'price']


In [234]:
predictions = knn.fit(train_no_prices, train_prices).predict(test_to_predict)
predictions = predictions.flatten()

In [235]:
#result = pd.DataFrame()
#result['price_usd'] = predictions
#result.index = test['id']
#result.to_csv('submission.csv')
real = properties[478:]
score = mean_squared_error(real['price'].values, predictions)
score

730740827175.60791

In [236]:
median_absolute_error(real['price'].values, predictions)

139948.0

#### Sin data subte
13133649304.42177

#### Con data subte
15440408563.133505

#### Con más data
4463304596.938776

3638297142.0196667

### Grid Search

In [95]:
testIds = properties.loc[478:]['id'].values

reducedProperties = properties[['id', 'price']]
reducedProperties.set_index(['id'], inplace=True)

k_trials = [1, 2, 3, 4, 5, 11, 21, 31, 51, 71, 91, 101, 151, 201, 251]
            
weights = ['uniform', 'distance']

algorithms = ['kd_tree', 'ball_tree']

predictionList = []
scoreList = []

for k in k_trials:
    for w in weights:
        for a in algorithms:
            knn = neighbors.KNeighborsRegressor(k, weights=w, algorithm=a)
            predictions = knn.fit(train_no_prices, train_prices).predict(test_to_predict)
            predictions = predictions.flatten()
            #Creo archivo para submit
            submissionDf = pd.DataFrame()
            submissionDf['id'] = pd.Series(testIds)
            submissionDf['price_usd'] = pd.Series(predictions)
            submissionDf.set_index(['id'], inplace=True)
            submissionDf.to_csv('predictions-K'+str(k)+'-'+w+'-'+a+'.csv')
            submissionDf.columns = ['price_usd']
            result = submissionDf.join(reducedProperties, how='inner')
            predictionList.append('KNN-K'+str(k)+'-'+w+'-'+a)
            real = properties[478:]
            score = mean_squared_error(real['price'].values, predictions)
            scoreList.append(score)

In [97]:
scoresDf = pd.DataFrame()
scoresDf['variant'] = pd.Series(predictionList)
scoresDf['score'] = pd.Series(scoreList)
scoresDf.sort_values(by='score', ascending=True, inplace=True)
scoresDf.to_csv('scores.csv')